## Analyze whether SNWD varies more from year to year or from place to place.

In [2]:
import pandas as pd
import numpy as np
import urllib
import math

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [5]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
420
+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|            coeff_1|            coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|               res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+-------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|-12268.765203613644| -825.7982574517629|2286.6759004873784|   2139.7|SSSBSSBB| 37.2275|-119.2206|       SNWD|0.08125103815927784|0.07298132861948865| 0.03728174536690476| 0

In [6]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [16]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


,station,year,coeff_3
0,USC00044176,1965.0,2286.675900
1,USC00049633,1973.0,-1863.413671
2,USC00044176,1953.0,901.552775
3,USC00049632,1967.0,1179.851233
4,USC00049632,1958.0,-207.004981


In [17]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station,USC00040684,USC00040755,USC00040820,USC00040824,USC00041697,USC00042331,USC00042539,USC00043939,USC00044170,USC00044176,...,USC00049855,USC00260046,USC00260668,USC00261755,USC00262431,USC00263285,USC00265362,USC00268160,USW00023153,USW00023157
year,,,,,,,,,,,,,,,,,,,,,
1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,518.016356,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.620809,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,587.161854,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1924.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1927.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1928.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1929.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,256.197785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,533.181161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [21]:
# coeff3
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station), \
        ', fraction explained = ', (1- MS(tbl_minus_station) /MS(year_station_table))*100
print 'MS removing mean-by-year   = ',MS(tbl_minus_year), \
        ', fraction explained = ', (1- MS(tbl_minus_year) /MS(year_station_table)) * 100

total MS                   =  1106973.62227
MS removing mean-by-station=  1038254.48275 , fraction explained =  6.20783893398
MS removing mean-by-year   =  482709.304796 , fraction explained =  56.3937843607


In [19]:
# coeff1
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station), \
        ', fraction explained = ', (1- MS(tbl_minus_station) /MS(year_station_table))*100
print 'MS removing mean-by-year   = ',MS(tbl_minus_year), \
        ', fraction explained = ', (1- MS(tbl_minus_year) /MS(year_station_table)) * 100

total MS                   =  1106973.62227
MS removing mean-by-station=  1038254.48275 , fraction explained =  6.20783893398
MS removing mean-by-year   =  482709.304796 , fraction explained =  56.3937843607


In [20]:
# coeff2
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station), \
        ', fraction explained = ', (1- MS(tbl_minus_station) /MS(year_station_table))*100
print 'MS removing mean-by-year   = ',MS(tbl_minus_year), \
        ', fraction explained = ', (1- MS(tbl_minus_year) /MS(year_station_table)) * 100

total MS                   =  1106973.62227
MS removing mean-by-station=  1038254.48275 , fraction explained =  6.20783893398
MS removing mean-by-year   =  482709.304796 , fraction explained =  56.3937843607


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1229.5296533
0 after removing mean by year    = 859.293116903
0 after removing mean by stations= 776.780909426
1 after removing mean by year    = 761.334597915
1 after removing mean by stations= 756.907401516
2 after removing mean by year    = 755.26569343
2 after removing mean by stations= 754.533930411
3 after removing mean by year    = 754.160786532
3 after removing mean by stations= 753.952127706
4 after removing mean by year    = 753.828256151
4 after removing mean by stations= 753.751890933
